In [1]:
import numpy as np
import pandas as pd
import os
import gc
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import xlearn as xl

In [4]:
train = np.genfromtxt('../../data/train_final_1/part-00000-c14a1c5f-edd9-42e4-8c9b-a52d5c6e2cf2-c000.csv', delimiter=',',)

In [5]:
train

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan,  0.,  7., ...,  0.,  2.,  1.],
       [nan,  0.,  7., ...,  0.,  4.,  5.],
       ...,
       [nan,  0.,  7., ...,  0.,  3.,  7.],
       [nan,  0.,  7., ...,  0.,  2.,  7.],
       [nan,  0.,  7., ...,  0.,  3.,  5.]])

In [3]:
path= '../../data/train_final_1/*.csv'

In [2]:
train = pd.read_csv('../../data/train_final_1/part-00000-c14a1c5f-edd9-42e4-8c9b-a52d5c6e2cf2-c000.csv')

In [3]:
train.head()

,MachineIdentifier,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,...,OsBuildLab_difftotal,DateAvSigVersion_difftotal,DateAvSigVersion_fulldifftotal,OsBuildLab_fulldifftotal,DateAvSigVersion_ratio,OsBuildLab_ratio,DateAvSigVersion_fullratio,OsBuildLab_fullratio,OsBuildLab_dayOfWeek,AvSigVersion_dayOfWeek
0,00001a027a0ab970c408182df8484fce,0,7,0,0,53447,1,1,1,164,...,224,76,76,224,0.0,0.0,0.0,0.0,2,1
1,000039104d7f42c15fdc0d1bcb9d4b4a,0,7,0,1950,53447,1,1,1,43,...,418,93,93,418,0.0,0.0,0.0,0.0,4,5
2,0000422df51e713f82b98e04cb883cb5,0,7,0,0,53447,1,1,1,9,...,418,100,100,418,0.0,0.0,0.0,0.0,4,5
3,0000bae35fb1dbde97ae59e962f24557,0,7,0,0,53447,1,1,1,57,...,219,97,97,219,0.0,0.0,0.0,0.0,7,1
4,0000db2c81925a9ec8d83e40be5ac070,0,7,0,0,53447,1,1,1,142,...,224,106,106,224,0.0,0.0,0.0,0.0,2,6


In [4]:
train.shape

(557684, 92)

In [5]:
sel_cols = [c for c in train.columns if c not in ['MachineIdentifier', 'HasDetections']]

In [6]:
X_train = train[sel_cols]
y_train = train['HasDetections']

In [7]:
train_ids = X_train.index
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
skf.get_n_splits(train_ids, y_train)

5

In [8]:
linear_model = xl.LRModel(task='binary', init=0.1,
                          epoch=50, lr=0.2,
                          reg_lambda=0.01, opt='sgd',
                          metric='auc', nthread=8,
                         stop_window=3)

In [9]:
ffm_model = xl.FFMModel(task='binary', 
                        lr=0.2, 
                        epoch=10, 
                        reg_lambda=0.002,
metric='auc')

In [18]:
fm_model = xl.FMModel(task='binary', init=0.5, 
                      epoch=100, k=10, lr=0.1, 
                      reg_lambda=0.01, opt='sgd', nthread=8, fold=5, stop_window=10, metric='auc',
                     block_size=10000)

In [19]:
counter = 0
for train_index, test_index in skf.split(train_ids, y_train):
    
    print('Fold {}\n'.format(counter + 1))
    
    X_fit, X_val = X_train.iloc[train_index, :], X_train.iloc[test_index, :]
    y_fit, y_val = y_train.iloc[train_index], y_train.iloc[test_index]
    
    fm_model.fit(X_fit, y_fit,
                 eval_set=[X_val, y_val])
                  
    del X_fit, X_val, y_fit, y_val, train_index, test_index
    gc.collect()
    
    counter += 1

Fold 1



KeyboardInterrupt: 

In [14]:
fm_model.get_params

<bound method BaseXLearnModel.get_params of FMModel(alpha=1, beta=1, block_size=500, epoch=50, fold=1, init=0.1, k=10,
    lambda_1=1, lambda_2=1, lr=0.1, metric='auc', model_type='fm',
    n_jobs=4, nthread=8, opt='sgd', reg_lambda=0.01, stop_window=2,
    task='binary')>

In [17]:
linear_model.fit(X_train, y_train,
                 eval_set=[X_test, y_test],
                 is_lock_free=False)

In [18]:
y_pred = linear_model.predict(X_test)

In [19]:
y_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [4]:
ffm_model = xl.create_ffm()                # Use field-aware factorization machine (ffm)
ffm_model.setTrain('/home/errodringer/Escritorio/Microsoft/data/train_final_1/part-00000-c14a1c5f-edd9-42e4-8c9b-a52d5c6e2cf2-c000.csv')    # Path of training data

# param:
#  0. task: binary classification
#  1. learning rate : 0.2
#  2. regular lambda : 0.002
param = {'task':'binary', 'lr':0.2, 'lambda':0.002}

In [ ]:
ffm_model.fit(param, "./model.out")